# Introduction

In the Part2 of this workshop, we will focus on model performance monitoring & feedback for the news classification model that we deployed

1. We will download and parse the training dataset, logs from prediction service that record inference traffic and annotations (ground truth labels for the inference traffic)
2. We will set up basic monitoring for system health (traffic volume, latency, SLA violations)
3. We will compute data and label drift for the inference traffic using a few different techniques (Chi-square statistic, KS-statistic, classifier-based drift detection)
4. We will understand model performance as a function of time for the inference traffic, and any ties we can derive to detected drift



In [1]:
import json
import numpy as np
import pandas as pd
from datetime import date, timedelta
import matplotlib.pyplot as plt

In [2]:
# Global Constants

LABEL_SET = [
    'Business',
    'Sci/Tech',
    'Software and Developement',
    'Entertainment',
    'Sports',
    'Health',
    'Toons',
    'Music Feeds'
]

DATA_URL = 's3://corise-mlops/qconsf/dataset_part2.zip'

LOG_DATE_START = date(2022, 10, 1)
LOG_DATE_END = date(2022, 10, 14)


## Download and Load Dataset

In Part1, we worked with a modified version of the [AG News](http://groups.di.unipi.it/~gulli/AG_corpus_of_news_articles.html) dataset - a collection of more than 1 million news articles gathered from more than 2000 news sources by an academic news search engine. This news topic classification dataset & benchmark was first used in [Character-level Convolutional Networks for Text Classification (NIPS 2015)](https://arxiv.org/abs/1509.01626). 

We started logging model inputs and predictions for the web application that we created for the trained news classifier. 

In Part2, We will work with the following files in the downloaded data:
1. `training.json` -- this is the training data on which the classification model was trained. This will act as the reference dataset when we want to compute things like drift and outliers. Each row in this file is a training data point.
2. `logs.json` -  this is a collection of logged outputs from our online service (the inference traffic). Each row in this file is a timestamped request, and contains the input request (text description, embedding, url etc) as well as model predictions. This will act as the target dataset when we want to compute things like drift and outliers. **The logs span a two week period from 2022/10/01 to 2022/10/14.**
3. `annotations.json` - this is the set of ground truth labels for requests received by our online prediction service. Imagine we have a team of human annotators that label a fraction of our inference stream (with some delay). Eventually these ground truth labels are logged and will be helpful for us to monitor online model performance, and also is a good source of future training data for the model. 

Sample row in the logs file:

```
{
    "id": 69265,
    "timestamp": "2022-10-11 00:00:00",
    "host_id": "host_4",
    "latency_ms": 169.676,
    "request": {
        "id": 69265,
        "source": null,
        "title": "Google May Face Another Lawsuit",
        "url": "http://www.pcworld.com/news/article/0,aid,117686,00.asp",
        "rank": "5",
        "description": "A federal judge in Virginia has ruled that a trademark infringement suit filed by the Government Employees Insurance Co. (GEICO) against Internet search giants Google and Overture Services can proceed.",
        "embedding": [...]
    },
    "pred_label": "Business",
    "pred_score": {
        "Business": 0.39581484916169474,
        "Entertainment": 0.19195937955028541,
        "Health": 0.02007952252798203,
        "Music Feeds": 0.0005983183076385058,
        "Sci/Tech": 0.3849374113779283,
        "Software and Developement": 0.001389120851963045,
        "Sports": 0.004566606342231832,
        "Toons": 0.0006547918802761208
    }
}
```
Sample row in the annotations file (rows in here should be joined to the correct request in `logs.json` using the "id" field):
```
{
    "id": 69265,
    "label": "Sci/Tech"
}
```

In [ ]:
from urllib.request import urlopen
from io import BytesIO
from zipfile import ZipFile


def download_dataset():
    """
    Download the dataset. The zip contains three files: train.json, test.json and unlabeled.json 
    """
    http_response = urlopen(DATA_URL)
    zipfile = ZipFile(BytesIO(http_response.read()))
    zipfile.extractall(path='data')

# Expensive operation so we should just do this once
download_dataset()


In [ ]:

training = pd.read_json('data/training.json', lines=True)
logs = pd.read_json('data/logs.json', lines=True)
true_labels = pd.read_json('data/annotations.json', lines=True)

In [ ]:
# examine the training dataset
training.head()

In [ ]:
# examine the logged traffic
logs.head()


## System Health & Metrics

Software system failures and downtimes are remarkably common even in ML systems. In an [analysis](https://www.youtube.com/watch?v=hBMHohkRgAA) conducted by Daniel Papasian and Todd Underwood (both ML engineers at Google), they looked at large ML pipeline failures at Google and found out that in 60 out of the 96 cases, the cause for failure was not directly related to the ML model.

As a start, tracking system health can be a good first step to ensure properties such as whether the deployed model is available online, is its latency within acceptable SLAs, are the system resources (such as CPU and memory usage) within bounds etc.


In [ ]:
"""
[TO BE IMPLEMENTED]

Each row in the logs.json file has an associated timestamp of the format YYYY-MM-DD HH:MM:SS.
We will use the timestamp to group requests by date, and track a few system metrics of interest:

1. Volume: Compute the daily volume of requests received by our service, grouped by host id.
2. Latency: Compute the mean, median, P90, P95 latency of requests received by our service, grouped by host id
3. Plot each of these as a line chart, with date on the X-axis and the computed metric on the Y-axis. 
You can plot the metric for each host id in the same line chart (with different colors for each host).

Do you notice anything strange with one of the hosts? :) 

"""


In [ ]:
"""
[TO BE IMPLEMENTED]

Let us say that we have a maximum latency SLA of 300 milliseconds i.e. 
if the prediction service took more than 300 ms to answer the request, the downstream experience is degraded for users.

1. Compute the aggregate daily volume of latency SLA violations (i.e. number of requests that have a latency >= maximum allowed latency) grouped by each host id 
2. Plot this metric as a line chart, with date on the X-axis and the computed metric on the Y-axis.
"""


## Monitoring model inputs and outputs

In supervised machine learning settings, we assume that the training dataset D = {X, y} is composed of input/label tuples {Xi, yi} that are independently drawn from some underlying joint distribution ℙ(X, y) such that ℙ(X, y) = ℙ(y|X)ℙ(𝐗)
 
ℙ(y|X) is the relationship we are trying to learn during the model training step, which can then be used to generate accurate predictions for unseen samples. We make two assumptions here:
1. The unseen samples that the model will be used to make predictions on, comes from the same underlying distribution ℙ(X, y).
2. This distribution ℙ(X, y) is stationary and does not change with time.

In practice, this assumption does not hold in most cases. Can we track and quantify this change over time though? 

## Input drift (hypothesis testing)

Hypothesis testing is a principled approach to evaluating drift. It’s a test to determine whether the difference between two populations (two sets of data) is statistically significant. If the difference is statistically significant, then the probability that the difference is a random fluctuation due to sampling variability is very low, and therefore, the difference is caused by the fact that these two populations come from two distinct distributions.

1. Design a test statistic (or a distance metric) that is computed on samples collected form the two distributions - in our case, the reference and target distributions (i.e. data points from the inference and reference datasets)
2. The test statistic is expected to be small if the null hypothesis is true (i.e. Z and Zref are drawn from the same distribution), and large if the alternative hypothesis (i.e. Z and Zref are drawn from different distributions) is true. 
3. From the test statistic, we compute a p-value: When p-value ≤ threshold, results from the test are said to be statistically significant, and the null hypothesis P(z) = Pref(z) is rejected. 


In [ ]:
"""
[TO BE IMPLEMENTED]

News source (e.g. New York Times, Reuters, BBC etc) is an important piece of metadata about the incoming requests.
Different news sources cover different topics, and any shift in the distribution/prevalence of news sources 
can be an important early signal to suggest that online traffic patterns might be changing. 

1. Both, the training dataset and infernce logs contains the news source metadata. 
2. Using the training dataset as the reference, quantify the drift in news sources for incoming requests. 
   You will compute drift for each day, using the inference logs from that day (i.e. "target") and comparing it to the training dataset (i.e. "reference")
3. Plot this metric as a line chart, with date on the X-axis and the computed metric on the Y-axis.

Chi-squared statistic is commonly used to measure drift for categorical features as we discussed.

Refer to: 
1) https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html 
2) https://docs.seldon.io/projects/alibi-detect/en/stable/cd/methods/chisquaredrift.html (in turn uses scipy under the hood)

These implementations will return the distance, and the p-value of the null hypothesis as part of the response
You want to plot the p-value in the line chart.

MAKE SURE YOU DEAL WITH MISSING/NULL/NONE VALUES IN THE TRAINING AND INFERENCE DATA IN A CONSISTENT WAY
"""

news_sources_ref = [row['source'] for row in training]
delta = LOG_DATE_END - LOG_DATE_START

news_sources_drift = []

for d in range(delta.days + 1):
   curr_date = LOG_DATE_START + timedelta(days=d)
   print(curr_date)

   # Step 1: collect all requests from logs.json with timestamp on the curr_date
   # this is currently a dummy empty list
   news_sources_target = []

   # Step 2: compute the drift between `news_sources_target` and `news_sources_ref` using Chi squared statistic
   # you can use either the scipy.stats.chisquare or the implementation in alibi-detect as shared in the references above

   # Step 3: store the pvalue in `news_sources_drift`


# Step 4:  Plot the `news_sources_drift` as a line chart, with date on the X-axis and the computed metric on the Y-axis.


In [ ]:
import matplotlib.pyplot as plt

x = [] # dates
y = [] # per day drift value

plt.plot(x, y, 'bo:')
plt.xlabel("date")
plt.xticks(rotation=90)
plt.show()


## Input drift (Classifier based)

The previous approach of treating drift detection as a two-sample hypothesis test is based on designing the correct test statistic. Oftentimes it can be hard to choose the correct statistic, and most test statistics are prone to false positives especially for multivate high dimensional data. 

What if we instead treat this as a classification problem? Can we train a classifier to predict which of the two distributions (reference or the target distribution) a given data point came from? The basic intuition is that if a classifier can learn to discriminate between the two distributions significantly better than random, then drift must have occurred.

The classifier-based drift detector tries to correctly distinguish instances from the reference dataset vs. the target dataset by training a classifier. It is possible to consume outputs of the classifier in a few different ways. 

1. We can binarize the classifier prediction score based on a decision threshold, and apply a binomial test on the binarized predictions of the reference vs. the target data.
2. We can use the classifier prediction score directly and compare the different in score distributions for the reference and target datasets.



In [ ]:
""" 
[TO BE IMPLEMENTED]

Embeddings represent an encoding of the semantic content of the inputs to the model, and also the feature space on 
top of which we train the classifier. Any shift in the space of embeddings can be an important signal to suggest 
that online traffic patterns might be changing which can impact downstream model performance

1. Both, the training dataset and infernce logs contain embedding representations of the news article description.
2. Using the training dataset as the reference, quantify the drift in embeddings for incoming requests. 
   You will compute drift for each day, using the inference logs from that day (i.e. "target") and comparing it to the training dataset (i.e. "reference")
3. Plot this metric as a line chart, with date on the X-axis and the computed metric on the Y-axis.

We will implement classifer-based drift detection for embeddings. You are free to choose the architecture of the classifier, 
but feel free to go with something simple (e.g. logistic regression, or a neural network with one hidden layer).

Remember, this is a binary classifier that tries to predict whether a given data point belongs to the reference or the target distribution. 

Also, remember that a new instance of the classifier will be trained for each (referce, target) pair i.e. for each day

Refer to: 
1) https://docs.seldon.io/projects/alibi-detect/en/stable/cd/methods/classifierdrift.html
2) https://openreview.net/forum?id=SJkXfE5xx 

These implementations will return the distance, and the p-value of the null hypothesis as part of the response
You want to plot the p-value in the line chart.
"""

from sklearn.linear_model import LogisticRegression


embeddings_ref = [row['embedding'] for row in training]

delta = LOG_DATE_END - LOG_DATE_START
embeddings_drift = []

for d in range(delta.days + 1):
   curr_date = LOG_DATE_START + timedelta(days=d)
   print(curr_date)

   # Step 1: collect all requests from logs.json with timestamp on the curr_date
   # this is currently a dummy empty list
   embeddings_target = []

   # Step 2: compute the drift between `embeddings_target` and `embeddings_ref`
   # (i) Initialize a new instance of the drift model (e.g. LogisticRegression())
   # (ii) Initialize the drift detector (check out https://docs.seldon.io/projects/alibi-detect/en/stable/cd/methods/classifierdrift.html)
   # (iii) use the initialized drift detector to compute the p-value

   # Step 3: store the pvalue in `embeddings_drift`


# Step 4:  Plot the `embeddings_drift` as a line chart, with date on the X-axis and the computed metric on the Y-axis.


In [ ]:
import matplotlib.pyplot as plt

x = []  # dates
y = []  # per day drift value

plt.plot(x, y, 'bo:')
plt.xlabel("date")
plt.xticks(rotation=90)
plt.show()


In [ ]:
""" 
[TO BE IMPLEMENTED]

It seems like there is some divergence in the distributions of embeddings between the reference and target set on some days. 
Let's visualize with UMAP (https://umap-learn.readthedocs.io/en/latest/). Uniform Manifold Approximation and Projection (UMAP) 
is a dimension reduction technique that can be used for visualisation similarly to t-SNE. 

1. We will train a UMAP model to project our original embedding space into 2 dimensions, using our reference (training) dataset.
2. We will then use this trained model to map the reference and the target dataset into two dimensions, and visualize it as a scatter plot. 

Do this exercise separately for two target distributions:
1. inference traffic from the first day
2. inference traffic from the last day
"""

import umap

def train_umap_model(emb):
    umap_model = umap.UMAP(
        n_components=2,
        n_neighbors=5,
        min_dist=0.1,
        spread=2.0
    )
    umap_model.fit(emb)
    return umap_model

In [ ]:
# Train UMAP on the reference data
embeddings_ref = [row['embedding'] for row in training]
umap_model = train_umap_model(embeddings_ref)
umap_ref = umap_model.transform(embeddings_ref)

# UMAP model inference
logs_10_01 = logs[logs['date'] == LOG_DATE_START]['request'].tolist()
embeddings_10_01 = [log['embedding'] for log in logs_10_01]
umap_10_01 = umap_model.transform(embeddings_10_01)

logs_10_14 = logs[logs['date'] == LOG_DATE_END]['request'].tolist()
embeddings_10_14 = [log['embedding'] for log in logs_10_14]
umap_10_14 = umap_model.transform(embeddings_10_14)

In [ ]:
from random import sample
import plotly.express as px

final_df = []

for idx, ut in enumerate(sample(umap_ref.tolist(), 2000)):
    final_df.append([ut[0], ut[1], 'reference'])

for idx, ut in enumerate(sample(umap_10_01.tolist(), 2000)):
    final_df.append([ut[0], ut[1], 'target(10/01)'])

df = pd.DataFrame(
    final_df, columns=["x", "y", "label"]
)

fig = px.scatter(df, x="x", y="y", color="label")
fig.update_traces(marker={'size': 3})
fig.show()


In [ ]:
from random import sample
import plotly.express as px

final_df = []

for idx, ut in enumerate(sample(umap_ref.tolist(), 2000)):
    final_df.append([ut[0], ut[1], 'reference'])

for idx, ut in enumerate(sample(umap_10_14.tolist(), 2000)):
    final_df.append([ut[0], ut[1], 'target(10/01)'])

df = pd.DataFrame(
    final_df, columns=["x", "y", "label"]
)

fig = px.scatter(df, x="x", y="y", color="label")
fig.update_traces(marker={'size': 3})
fig.show()


## Output (Label) Drift

In [ ]:
""" 
[TO BE IMPLEMENTED]

In this section, we will repeat the analysis we did in Step 4 [part 1], but for model predictions (outputs). 

A shift in the distribution of predicted labels and/or scores can be an important signal to suggest 
that online traffic patterns might be changing. 

1. The infernce logs contains the `pred_label` which is the predicted label of the model
2. Using true labels from training dataset as the reference, quantify label drift for incoming requests. 
   You will compute drift for each day, using the inference logs from that day (i.e. "target") and comparing it to the training dataset (i.e. "reference")
3. Plot this metric as a line chart, with date on the X-axis and the computed metric on the Y-axis.

Chi-squared statistic is commonly used to measure drift for categorical features as we discussed.

Refer to: 
1) https://docs.scipy.org/doc/scipy/reference/generated/scipy.stats.chisquare.html 
2) https://docs.seldon.io/projects/alibi-detect/en/stable/cd/methods/chisquaredrift.html (in turn uses scipy under the hood)

These implementations will return the distance, and the p-value of the null hypothesis as part of the response
You want to plot the p-value in the line chart.

Which labels are the most over-predicted compared to their prevalence in the training data? 
Which labels are the most under-predicted compared to their prevalence in the training data? 
"""


## Model performance

Ground truth observations (true labels) are a critical part of all supervised/self-supervised machine learning. Ground truth is important not just for model training, but also for monitoring and measuring performance of models after deployment.

`annotations.json` is the set of ground truth labels for requests received by our online prediction service. Imagine we have a team of human annotators that label a fraction of our inference stream (with some delay). Eventually these ground truth labels are logged and will be helpful for us to monitor online model performance.


In [ ]:
"""
[TO BE IMPLEMENTED]

Weekly classification report

Compute the a classification report for each week of the logged inference data
   Week 1 = [2022-10-01 and 2022-10-07]
   Week 2 = [2022-10-08 and 2022-10-14]

Do you notice a trend in model performance? 
How does this correlate to observed drift in news source, embeddings etc ? 
"""

from sklearn import metrics as sklearn_metrics
print(sklearn_metrics.classification_report([], []))

In [ ]:
"""
[TO BE IMPLEMENTED]

Daily aggregate classification accuracy

1. Compute the daily aggregate classification accuracy of the model
2. Plot this metric as a line chart, with date on the X-axis and the computed metric on the Y-axis. 

Do you notice a trend in model performance? 
How does this correlate to observed drift in news source, embeddings etc ? 
"""
